<a href="https://colab.research.google.com/github/IMINTP/Splines/blob/main/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
import zipfile
import os

# 압축 파일 경로
zip_path = "/content/drive/MyDrive/converted_frames.zip"

# 압축 해제할 경로 설정
extract_dir = "/content/drive/MyDrive/converted_frames_extracted"

# 압축 풀기
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("압축 해제 완료")


압축 해제 완료


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# CSV 경로 및 이미지 디렉토리 경로 설정
csv_path = "/content/drive/MyDrive/angle_data_float.csv"
image_dir = "/content/drive/MyDrive/converted_frames_extracted/converted_frames"

# CSV 데이터 로드
df = pd.read_csv(csv_path)

# 이미지 경로와 angle 데이터를 연결하는 함수 (재귀적으로 하위 폴더를 탐색)
def load_images_and_angles(row):
    file_path = row['file_path']
    images = []
    primary_angle = row['primary_angle']
    secondary_angle = row['secondary_angle']

    for frame_number in range(100):  # 최대 100프레임 가정
        # 프레임 번호에 맞는 파일 이름 생성
        image_name = file_path.split('/')[-1].replace('.dcm', f'_frame_{frame_number}.png')

        # 하위 폴더에서 해당 이미지 파일 찾기
        for root, dirs, files in os.walk(image_dir):
            if image_name in files:
                full_image_path = os.path.join(root, image_name)

                # 이미지 파일 로드
                img = load_img(full_image_path, target_size=(299, 299))
                img = img_to_array(img)
                img = tf.keras.applications.inception_v3.preprocess_input(img)
                images.append(img)
                break
        else:
            print(f"Image not found: {image_name}")

    return images, primary_angle, secondary_angle

# 이미지와 각도를 리스트로 저장
images = []
primary_angles = []
secondary_angles = []

# CSV 데이터에서 이미지 경로와 각도 데이터를 가져옴
for _, row in df.iterrows():
    img_list, primary_angle, secondary_angle = load_images_and_angles(row)
    if img_list:  # 이미지 리스트가 비어 있지 않은 경우
        images.extend(img_list)  # 모든 이미지 추가
        primary_angles.extend([primary_angle] * len(img_list))  # 각도 데이터를 이미지 수만큼 추가
        secondary_angles.extend([secondary_angle] * len(img_list))

# 데이터를 NumPy 배열로 변환
X = np.array(images)
y_primary = np.array(primary_angles)
y_secondary = np.array(secondary_angles)

# 로드된 이미지의 수 확인
print(f"Total images loaded: {X.shape[0]}")
print(f"Primary angles: {y_primary.shape[0]}, Secondary angles: {y_secondary.shape[0]}")


Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_0.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_1.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_2.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_3.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_4.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_5.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_6.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_7.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_8.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_9.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_frame_10.png
Image not found: toy_dataset\10xxxxxx\1012xxxx\10123450\20150318\XA\002_fra

KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# CSV 경로 및 이미지 디렉토리 경로 설정
csv_path = "/content/drive/MyDrive/angle_data_float.csv"
image_dir = "/content/drive/MyDrive/converted_frames_extracted/converted_frames"

# CSV 데이터 로드
df = pd.read_csv(csv_path)

# 이미지 경로와 angle 데이터를 연결하는 함수 (경로 구분자 문제 해결)
def load_images_and_angles(row):
    # CSV에서 이미지 파일 경로를 추출
    file_path = row['file_path']
    images = []
    primary_angle = row['primary_angle']
    secondary_angle = row['secondary_angle']

    for frame_number in range(100):  # 최대 100프레임 가정
        # 프레임 번호에 맞는 파일 이름 생성
        # '.dcm'을 '_frame_{frame_number}.png'로 대체
        image_name = file_path.split('/')[-1].replace('.dcm', f'_frame_{frame_number}.png')

        # 파일 경로를 적절히 설정 (경로 내에 있는 역슬래시(\)를 슬래시(/)로 변경)
        rel_path = file_path.replace("C:/python/angle/toy_dataset", "")  # 불필요한 toy_dataset 부분 제거
        rel_path = rel_path.replace("\\", "/")  # 경로 내의 역슬래시를 슬래시로 변경
        full_image_path = os.path.join(image_dir, rel_path).replace('.dcm', f'_frame_{frame_number}.png')

        # Colab 환경에서 경로 확인
        print(f"Checking image path: {full_image_path}")

        if os.path.exists(full_image_path):
            # 이미지를 InceptionV3 입력 크기에 맞게 로드하고 전처리
            img = load_img(full_image_path, target_size=(299, 299))
            img = img_to_array(img)
            img = tf.keras.applications.inception_v3.preprocess_input(img)
            images.append(img)
        else:
            print(f"Image not found: {full_image_path}")

    return images, primary_angle, secondary_angle

# 이미지와 각도를 리스트로 저장
images = []
primary_angles = []
secondary_angles = []

# CSV 데이터에서 이미지 경로와 각도 데이터를 가져옴
for _, row in df.iterrows():
    img_list, primary_angle, secondary_angle = load_images_and_angles(row)
    if img_list:  # 이미지 리스트가 비어 있지 않은 경우
        images.extend(img_list)  # 모든 이미지 추가
        primary_angles.extend([primary_angle] * len(img_list))  # 각도 데이터를 이미지 수만큼 추가
        secondary_angles.extend([secondary_angle] * len(img_list))

# 데이터를 NumPy 배열로 변환
X = np.array(images)
y_primary = np.array(primary_angles)
y_secondary = np.array(secondary_angles)

# 로드된 이미지의 수 확인
print(f"Total images loaded: {X.shape[0]}")
print(f"Primary angles: {y_primary.shape[0]}, Secondary angles: {y_secondary.shape[0]}")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Checking image path: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_1.png
Image not found: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_1.png
Checking image path: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_2.png
Image not found: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_2.png
Checking image path: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_3.png
Image not found: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_3.png
Checking image path: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_4.png
Image not found: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_4.png
Checking image path: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_5.png
Image not found: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_5.png
Checking image path: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_6.png
Image not found: /24xxxxxx/2492xxxx/24921350/20180512/XA/043_frame_6.png
Checking image path: /24xxxxxx/2492xxxx/24921350/20180512/XA/043

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# CSV 경로 및 이미지 디렉토리 경로 설정
csv_path = "/content/drive/MyDrive/angle_data_float.csv"
image_dir = "/content/drive/MyDrive/converted_frames_extracted/converted_frames"

# CSV 데이터 로드
df = pd.read_csv(csv_path)

# 이미지 경로와 angle 데이터를 연결하는 함수
def load_images_and_angles(row):
    # CSV에서 이미지 파일 경로를 추출
    file_path = row['file_path']
    images = []
    primary_angle = row['primary_angle']
    secondary_angle = row['secondary_angle']

    # 경로 처리: CSV에 있는 Windows 경로를 Colab 경로로 변환
    # toy_dataset이 없기 때문에, 해당 부분을 제거 및 경로를 맞춰줍니다.
    rel_path = file_path.replace("C:/python/angle/toy_dataset", "")
    rel_path = rel_path.replace("\\", "/")  # 역슬래시를 슬래시로 변경

    # 이미지 파일명은 .dcm을 제거하고 frame 번호로 변환
    for frame_number in range(100):  # 최대 100프레임 가정
        image_name = f'_frame_{frame_number}.png'
        full_image_path = os.path.join(image_dir, rel_path.lstrip("/")).replace('.dcm', image_name)

        # Colab 환경에서 경로 확인
        print(f"Checking image path: {full_image_path}")

        if os.path.exists(full_image_path):
            # 이미지를 InceptionV3 입력 크기에 맞게 로드하고 전처리
            img = load_img(full_image_path, target_size=(299, 299))
            img = img_to_array(img)
            img = tf.keras.applications.inception_v3.preprocess_input(img)
            images.append(img)
        else:
            print(f"Image not found: {full_image_path}")

    return images, primary_angle, secondary_angle

# 이미지와 각도를 리스트로 저장
images = []
primary_angles = []
secondary_angles = []

# CSV 데이터에서 이미지 경로와 각도 데이터를 가져옴
for _, row in df.iterrows():
    img_list, primary_angle, secondary_angle = load_images_and_angles(row)
    if img_list:  # 이미지 리스트가 비어 있지 않은 경우
        images.extend(img_list)  # 모든 이미지 추가
        primary_angles.extend([primary_angle] * len(img_list))  # 각도 데이터를 이미지 수만큼 추가
        secondary_angles.extend([secondary_angle] * len(img_list))

# 데이터를 NumPy 배열로 변환
X = np.array(images)
y_primary = np.array(primary_angles)
y_secondary = np.array(secondary_angles)

# 로드된 이미지의 수 확인
print(f"Total images loaded: {X.shape[0]}")
print(f"Primary angles: {y_primary.shape[0]}, Secondary angles: {y_secondary.shape[0]}")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Image not found: /content/drive/MyDrive/converted_frames_extracted/converted_frames/20xxxxxx/2075xxxx/20751249/20140224/XA/016_frame_91.png
Checking image path: /content/drive/MyDrive/converted_frames_extracted/converted_frames/20xxxxxx/2075xxxx/20751249/20140224/XA/016_frame_92.png
Image not found: /content/drive/MyDrive/converted_frames_extracted/converted_frames/20xxxxxx/2075xxxx/20751249/20140224/XA/016_frame_92.png
Checking image path: /content/drive/MyDrive/converted_frames_extracted/converted_frames/20xxxxxx/2075xxxx/20751249/20140224/XA/016_frame_93.png
Image not found: /content/drive/MyDrive/converted_frames_extracted/converted_frames/20xxxxxx/2075xxxx/20751249/20140224/XA/016_frame_93.png
Checking image path: /content/drive/MyDrive/converted_frames_extracted/converted_frames/20xxxxxx/2075xxxx/20751249/20140224/XA/016_frame_94.png
Image not found: /content/drive/MyDrive/converted_frames_extracted/converted_frames/20xxxxxx/2075xxxx/20751249/2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Train/Test 데이터 분리 (80% train, 20% test)
X_train, X_test, y_train_primary, y_test_primary, y_train_secondary, y_test_secondary = train_test_split(
    X, y_primary, y_secondary, test_size=0.2, random_state=42)

# InceptionV3 모델 로드 (pre-trained 모델 사용, 최상위 레이어 제외)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# InceptionV3의 출력 레이어 가져오기
x = base_model.output
x = GlobalAveragePooling2D()(x)  # 전역 평균 풀링을 적용하여 출력 차원을 줄임

# Primary Angle 예측 레이어
primary_angle_output = Dense(1, activation='linear', name='primary_angle')(x)

# Secondary Angle 예측 레이어
secondary_angle_output = Dense(1, activation='linear', name='secondary_angle')(x)

# 모델 정의 (두 개의 출력)
model = Model(inputs=base_model.input, outputs=[primary_angle_output, secondary_angle_output])

# Pre-trained InceptionV3 레이어를 학습하지 않도록 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일 (각도 예측이므로 MSE 손실 함수와 Adam 옵티마이저 사용)
model.compile(optimizer='adam',
              loss={'primary_angle': 'mean_squared_error', 'secondary_angle': 'mean_squared_error'},
              metrics={'primary_angle': 'mae', 'secondary_angle': 'mae'})  # MAE 메트릭 사용

# 모델 학습
history = model.fit(X_train,
                    {'primary_angle': y_train_primary, 'secondary_angle': y_train_secondary},
                    validation_data=(X_test, {'primary_angle': y_test_primary, 'secondary_angle': y_test_secondary}),
                    epochs=10,  # 10 에포크로 학습 진행 (필요 시 조정 가능)
                    batch_size=16)  # 배치 크기를 16으로 설정

# 모델 평가
loss, primary_mae, secondary_mae = model.evaluate(X_test,
                                                  {'primary_angle': y_test_primary, 'secondary_angle': y_test_secondary})
print(f"Test loss: {loss}")
print(f"Test Primary Angle MAE: {primary_mae}")
print(f"Test Secondary Angle MAE: {secondary_mae}")


87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/10
349/349 [==============================] - 113s 312ms/step - loss: 940.5533 - primary_angle_loss: 525.2862 - secondary_angle_loss: 415.2674 - primary_angle_mae: 20.0748 - secondary_angle_mae: 16.9919 - val_loss: 788.4537 - val_primary_angle_loss: 424.7245 - val_secondary_angle_loss: 363.7293 - val_primary_angle_mae: 17.7718 - val_secondary_angle_mae: 15.9729
Epoch 2/10
349/349 [==============================] - 106s 304ms/step - loss: 689.7666 - primary_angle_loss: 353.3556 - secondary_angle_loss: 336.4111 - primary_angle_mae: 15.8792 - secondary_angle_mae: 15.2968 - val_loss: 619.8883 - val_primary_angle_loss: 314.7849 - val_secondary_angle_loss: 305.1032 - val_primary_angle_mae: 14.8930 - val_secondary_angle_mae: 14.5599
Epoch 3/10
349/349 [==============================] - 105s 302ms/step - loss: 560.2798 - primary_angle_loss: 275.2894 - secondary_angle_loss: 284.9901 - primary_angle_mae: 13.6299 - secondary

ValueError: too many values to unpack (expected 3)

In [ ]:
# 모델 저장
model.save('/content/drive/MyDrive/saved_model/my_inception_model.h5')


NameError: name 'model' is not defined